In [53]:
import dicom
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import scipy.ndimage
import cv2

INPUT_FOLDER = 'sample_images (1)/'
patients = os.listdir(INPUT_FOLDER)
labels_df = pd.read_csv('stage1_labels.csv', index_col=0)


In [2]:
label_new = np.zeros(20)
for i in range(len(patients)):
    try:
        label= labels_df.get_value(patients[i], 'cancer')
        
    except KeyError as e:
        label = 0
    
    label_new[i] = label
    

In [67]:
# load dicom files and add slice thickness

def load_slices(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try: 
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    
    slices1 = []
    
    for s in slices:
        s.SliceThickness = slice_thickness
    
    return slices

In [70]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    image = image.astype(np.int16)

    image[ image == -2000] = 0

    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
    
        if slope!=1:
            image[slice_number] = slope*image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            image[slice_number] += np.int16(intercept)
        
    
    return np.array(image, dtype = np.int16)


In [5]:
def resample(image, scan, new_spacing):
    spacing = np.array([scan[0].SliceThickness] + scan[0].PixelSpacing, dtype=np.float32)
    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode = 'nearest')
    
    return image, new_spacing
    

In [6]:
MIN_BOUND = -1000.0
MAX_BOUND = 400.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

In [7]:
PIXEL_MEAN = 0.25

def zero_center(image):
    image = image - PIXEL_MEAN
    return image

In [86]:
slices = load_slices(INPUT_FOLDER + patients[10])
image = get_pixels_hu(slices)
image, spacing = resample(image, slices, [1,1,1])

slice2 = []
for sliceee in image:
    slice1 = cv2.resize(sliceee, (25,25))
    slice2.append(slice1)


In [97]:
import math
HM_SLICES = 20
new_slices = []

image = np.stack([s for s in slice2])

np.shape(image)

def mean(l):
    return sum(l) / len(l)

def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

chunk_sizes = math.ceil(len(slice2) / HM_SLICES)
for slice_chunk in chunks(slice2, chunk_sizes):
    slice_chunk = list(map(mean, zip(*slice_chunk)))
    new_slices.append(slice_chunk)

np.shape(new_slices)

(20, 25, 25)

In [124]:
from tqdm import tqdm

much_data = []
new_label = np.zeros(20)
for i in range(20):
    
    try:
        label= labels_df.get_value(patients[i], 'cancer')
        
    except KeyError as e:
        label = 0
    new_label[i] = label
    
    slices = load_slices(INPUT_FOLDER + patients[i])
    image = get_pixels_hu(slices)
    image, spacing = resample(image, slices, [1,11,1])
    slice2 = []
    new_slices = []
    
    for sliceee in image:
        slice1 = cv2.resize(sliceee, (50,50))
        slice2.append(slice1)
        
    chunk_sizes = math.ceil(len(slice2) / HM_SLICES)
    for slice_chunk in chunks(slice2, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)
    
    if len(new_slices) == HM_SLICES-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == HM_SLICES-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == HM_SLICES+2:
        new_val = list(map(mean, zip(*[new_slices[HM_SLICES-1],new_slices[HM_SLICES],])))
        del new_slices[HM_SLICES]
        new_slices[HM_SLICES-1] = new_val
        
    if len(new_slices) == HM_SLICES+1:
        new_val = list(map(mean, zip(*[new_slices[HM_SLICES-1],new_slices[HM_SLICES],])))
        del new_slices[HM_SLICES]
        new_slices[HM_SLICES-1] = new_val
        
    
    image = np.stack([s for s in new_slices])
    
    image = normalize(image)
    image = zero_center(image)
    
    much_data.append([image, label])
     
    

In [125]:

for i in range(20):
    print(np.shape(much_data[i][0]))

(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)
(20, 50, 50)


In [126]:
import tensorflow as tf

x = tf.placeholder('float')
y = tf.placeholder('float')
n_classes = 2
batch_size = 10
keep_rate = 0.8

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides = [1,1,1,1,1], padding = 'SAME')

def maxpool3d(x):
    return tf.nn.max_pool3d(x, ksize = [1,2,2,2,1], strides = [1,2,2,2,1], padding = 'SAME')

def convolutional_3d(x):
    
    weights = {'W_conv1':tf.Variable(tf.truncated_normal([3,3,3,1,32], stddev = 0.1)),
               'W_conv2':tf.Variable(tf.truncated_normal([3,3,3,32,64], stddev = 0.1)),
               'W_fc':tf.Variable(tf.truncated_normal([54080,1024], stddev = 0.1)),
               'out':tf.Variable(tf.truncated_normal([1024, n_classes], stddev = 0.1))}
    
    biases = { 'b_conv1':tf.Variable(tf.constant(0.1, shape = [32])),
               'b_conv2':tf.Variable(tf.constant(0.1, shape = [64])),
               'b_fc':tf.Variable(tf.constant(0.1, shape = [1024])),
               'out':tf.Variable(tf.constant(0.1, shape = [n_classes]))}
    
    x = tf.reshape(x, shape=[-1, 50, 50, 20, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    #fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output
    

In [131]:
def train_neural_network(x):
    
    train_data = much_data[:-15]
    validation_data = much_data[-15:]
    
    
    prediction = convolutional_3d(x)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction, y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    sess= tf.Session()
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(15):
        
        for data in train_data:
            X = data[0]
            Y=data[1]
            sess.run(optimizer, feed_dict = {x: X, y:Y})
        
        train_accuracy = sess.run(accuracy, feed_dict={x:X, y: Y})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    
    print(sess.run(accuracy, feed_dict = {x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))


train_neural_network(x)

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ['Tensor("Variable/read:0", shape=(3, 3, 3, 1, 32), dtype=float32)', 'Tensor("Variable_1/read:0", shape=(3, 3, 3, 32, 64), dtype=float32)', 'Tensor("Variable_2/read:0", shape=(8800, 1024), dtype=float32)', 'Tensor("Variable_3/read:0", shape=(1024, 2), dtype=float32)', 'Tensor("Variable_4/read:0", shape=(32,), dtype=float32)', 'Tensor("Variable_5/read:0", shape=(64,), dtype=float32)', 'Tensor("Variable_6/read:0", shape=(1024,), dtype=float32)', 'Tensor("Variable_7/read:0", shape=(2,), dtype=float32)', 'Tensor("Variable_8/read:0", shape=(3, 3, 3, 1, 32), dtype=float32)', 'Tensor("Variable_9/read:0", shape=(3, 3, 3, 32, 64), dtype=float32)', 'Tensor("Variable_10/read:0", shape=(54080, 1024), dtype=float32)', 'Tensor("Variable_11/read:0", shape=(1024, 2), dtype=float32)', 'Tensor("Variable_12/read:0", shape=(32,), dtype=float32)', 'Tensor("Variable_13/read:0", shape